In [2]:
import numpy as np
from ueg import ueg_qc
from pyscf import gto, scf, ao2mo, cc
from ueg import my_ueg, ueg_qc

In [ ]:
nocc = 57 # 7, 19, 27, 33, 57, 81, 93
system = ueg_qc(1.0, (nocc, nocc), e_cut_red=5) # this e_cut is for ueg_qc().get_k_points()
k_points = system.get_k_points()
# my_sys = my_ueg(rs=1.0, nelec=(nocc, nocc))
# k_points = my_sys.get_kpts(np.sqrt(2)) # get k_pts by my_ueg, this allows k_cut = gamma*k_fermi
n_kpts = k_points.shape[0]
print(f"number of k-points: {n_kpts}")
h0 = system.madelung() / 2
h1 = system.get_h1_real(k_points)
eri = system.get_eri_tensor_real(k_points)
eri = np.array(eri)

mol = gto.M()
mol.nelectron = system.n_particles
mol.nao = n_kpts
mol.incore_anyway = True
mol.max_memory = 20000
# mol.energy_nuc = lambda *args: h0
mol.verbose = 4

# dm = np.zeros((n_kpts,n_kpts))
# dm[:nocc,:nocc] = np.eye(nocc) * 2.0

mf = scf.RHF(mol)
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(n_kpts)
# mf.get_jk = get_jk
mf.verbose = 4
mf._eri = ao2mo.restore(8, eri, n_kpts)
mf.init_guess = "1e"
mf.max_cycle = -1
# mf.mo_coeff = np.eye(n_kpts)
escf = mf.kernel()

# mf.mo_coeff = np.eye(n_kpts)

print(f"escf: {escf}")
print(f"escf per electron: {escf/system.n_particles}")

mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
# mycc1.conv_tol = 1e-7
mycc.kernel()
print("CCSD energy", mycc.e_tot)
print("CCSD E_corr per electron", mycc.e_corr / system.n_particles)
print("CCSD energy per electron", mycc.e_tot / system.n_particles)
# et_correction = mycc1.ccsd_t()
# print("CCSD(T) energy", mycc1.e_tot + et_correction)
# print("CCSD(T) energy per electron", (mycc1.e_tot + et_correction) / system.n_particles)

number of k-points: 147


In [4]:
def prep_afqmc(mycc,chol_cut=1e-6,
               amp_file="amplitudes.npz",
               chol_file="FCIDUMP_chol"):
    from pyscf import lib
    from ad_afqmc import pyscf_interface

    mf = mycc._scf
    mol = mf.mol
    nelec = mol.nelec
    nao = mol.nao
    
    t1 = np.array(mycc.t1)
    t2 = mycc.t2
    t2 = t2.transpose(0, 2, 1, 3)
    np.savez(amp_file, t1=t1, t2=t2)

    # calculate cholesky integrals
    print("# Calculating Cholesky integrals")
    
    h0 = mf.energy_nuc()
    h1 = mf.get_hcore()
    h2 = mf._eri.reshape(nao**2,nao**2)
    chol = pyscf_interface.modified_cholesky(h2,chol_cut)
    chol = chol.reshape((-1, nao, nao))
    nchol = chol.shape[0]

    v0 = 0.5 * lib.einsum("gpr,grq->pq", chol, chol, optimize="optimal")
    h1_mod = h1 - v0
    chol = chol.reshape((chol.shape[0], -1))

    print("# Finished calculating Cholesky integrals#")
    print("# Size of the correlation space:")
    print(f"# Number of electrons: {nelec}")
    print(f"# Number of basis functions: {nao}")
    print(f"# Number of Cholesky vectors: {nchol}")

    
    pyscf_interface.write_dqmc(
        h1,
        h1_mod,
        chol,
        sum(nelec),
        nao,
        h0,
        ms=0,
        filename=chol_file,
    )

    return None

In [5]:
prep_afqmc(mycc,chol_cut=1e-6)

# Calculating Cholesky integrals
# Finished calculating Cholesky integrals#
# Size of the correlation space:
# Number of electrons: (7, 7)
# Number of basis functions: 19
# Number of Cholesky vectors: 92


In [6]:
prep_afqmc(mycc,chol_cut=1e-6)
options = {'n_eql': 3,
           'n_prop_steps': 50,
            'n_ene_blocks': 1,
            'n_sr_blocks': 5,
            'n_blocks': 10,
            'n_walkers': 10,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'ccsd_pt',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc.prop_unrestricted import prop_unrestricted
prop_unrestricted.run_afqmc(options,nproc=1)

# Calculating Cholesky integrals
# Finished calculating Cholesky integrals#
# Size of the correlation space:
# Number of electrons: (7, 7)
# Number of basis functions: 19
# Number of Cholesky vectors: 92
# running AFQMC on CPU
# AFQMC script: /home/yichi/research/software/cs_afqmc/ad_afqmc/prop_unrestricted/run_afqmc_ccsd_pt.py
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 19
# nelec: (7, 7)
#
# n_eql: 3
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 5
# n_blocks: 10
# n_walkers: 10
# seed: 2
# walker_type: rhf
# trial: ccsd_pt
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#
# Propagating with 10 walkers
# Equilibration sweeps:
#   Iter 	 <t> 	 	